In [ ]:
using DifferentialEquations, ParameterizedFunctions
using JSON2
using Plots, Colors
using LaTeXStrings

In [ ]:
using Plots.Measures
font_title = Plots.font("Arial", 24)
font_axis = Plots.font("Arial", 12)
font_legend = Plots.font("Arial", 8)
pyplot(titlefont=font_title, guidefont = font_axis, legendfont = font_legend)

#Define different symbols used in the plotting
delta = latexstring("\$\\delta\$")
micro = latexstring("\$\\mu\$")
vt = latexstring("\$V_t\$")
nt = latexstring("\$N_t\$")
i_ach = latexstring("\$I_{ACh}\$")

v_color = :deepskyblue
n_color = :magenta
c_color = :green
a_color = :purple
b_color = :red
ach_color = :blue

In [ ]:
figure_path = "C:/users/mtarc/JuliaScripts/RetinalChaos/Notebooks/Figures"

In [ ]:
import RetinalChaos: read_JSON, extract_dict
import RetinalChaos: T_ode

In [ ]:
p_dict = read_JSON("params.json") 
p_dict[:I_app] = 10.0
p_dict[:g_ACh] = 0.0
p = p_dict |> extract_dict;
u0 = read_JSON("conds.json") |> extract_dict;
dt = 0.1
tspan = (0.0, 60e3);
prob = ODEProblem(T_ode, u0, tspan, p)
println("Time it took to simulate 60s:")
@time sol = solve(prob, abstol = 2e-2, reltol = 2e-2, maxiters = 1e7, saveat = dt); 

In [ ]:
plot(sol, vars = [:v])

### Figure 1 Time Scale Analysis

In [ ]:
#Making Figure part A
xlims = (23.8e3, 24e3)
ellapsed_time = (xlims[end]-xlims[1])
xticks = (collect(xlims[1]:20.0:xlims[2]), collect(0:20.0:ellapsed_time))
fig1_Aa = plot(sol, 
    vars = [:v, :n,], 
    ylabel = ["$vt (mV)" "$nt"], xlabel = ["" "time (ms)"], 
    lw = 2.0,
    c = [v_color n_color],
    legend = :none, 
    layout = grid(2, 1), 
    xlims = xlims,
    xticks = xticks
)
vt_sol = map(t -> sol(t)[1], collect(xlims[1]:0.1:xlims[2]))
nt_sol = map(t -> sol(t)[2], collect(xlims[1]:0.1:xlims[2]))
fig1_Ab = plot(vt_sol, nt_sol, c = :black, label = "", 
    title = "$(vt) to $(nt) Oscillation Diagram", titlefontsize = 12.0,
    xlabel = "$vt (mV)", ylabel = "$nt"
)

fig1_A = plot(fig1_Aa, fig1_Ab, layout = grid(1,2, widths = [0.7, 0.3]), grid = false);
title!(fig1_A[1], "A", title_location = :left)

In [ ]:
import RetinalChaos.M_INF

In [ ]:
v_rng = collect(-100:1.0:0)
C0 = p_dict[:C_0]
Ci = C0
δ = p_dict[:δ]
λ = p_dict[:λ]
#for calculating of I_ca
gCa = p_dict[:g_Ca]
ECa = p_dict[:E_Ca]
V1 = p_dict[:V1]
V2 = p_dict[:V2]
τc = p_dict[:τc]
f(v) = (C0+δ*(-gCa * M_INF(v, V1, V2) * (v - ECa)) - (λ*Ci))/τc
dc_rng = map(f, v_rng);

In [ ]:
#Figure 1 part B
xlims = (22e3, 26e3) 
xticks = (collect(xlims[1]:500:xlims[2]), collect(0:0.5:(xlims[1]-xlims[2]/1000)))
fig1_Ba = plot(sol, 
    vars = [:v, :c], 
    ylabel = ["$vt (mV)" "[Ca] (mM)"], 
    xlabel = ["" "time (s)"], 
    c = [v_color c_color], lw = 2.0,
    legend = :none, 
    layout = grid(2, 1), 
    xlims = xlims, 
    xticks = xticks
)

fig1_Bb = plot(v_rng, dc_rng*1000, 
    c = c_color, linestyle = :dash, lw = 2.0, fill = (0, 0.5, c_color),
    title = "Instantaneous Calcium Change", titlefontsize = 12.0,
    xlabel = "$vt (mV)",ylabel = "$(delta)[Ca] (mM/s)", label = "")

fig1_B = plot(fig1_Ba, fig1_Bb, 
    layout = grid(1, 2, widths=[0.7, 0.3]), size = (700, 350), grid = false)
title!(fig1_B[1], "B", title_pos = :left)

In [ ]:
c_rng = collect(0.0:0.01:1.5)
ai = 1.0
α = p_dict[:α]
τa = p_dict[:τa]
fa(c) = (-(α*c^4)/τa) + (1/τa)*ai
a_rng = map(fa, c_rng).*1000;
zero_c = (1/α)^(1/4)

a_rng_2 = collect(0.0:0.01:1.5)
bi = 0.0
β = p_dict[:β]
τb = p_dict[:τb]
fb(a) = (β*a^4)/τb - 1.0/τb
b_rng = -map(fb, a_rng_2).*1000
zero_a = (1/β)^(1/4)

In [ ]:
fig1_Ca = plot(layout = grid(4, 1, heights = [0.2, 0.2, 0.2, 0.4]))

xlims = (0.0, 40e3) 
xticks = (collect(xlims[1]:5e3:xlims[end]), collect(0:5.0:(xlims[end]-xlims[1]/1000)))

plot!(
    fig1_Ca[1], sol, vars = [:c], 
    ylabel = "[Ca] (mM)", xlabel = "",
    c = c_color, lw = 2.0, 
    legend = :none, 
    xlims = xlims, 
    xticks = xticks, 
    fill = (0.1, 0.2, a_color)
)

t_sub = (0:40e3)

a_m = map(t -> 1-sol(t)[4], collect(0:40e3))
plot!(
    fig1_Ca[2], t_sub, a_m, label = "",
    ylabel = "normal [cAMP]", c = a_color, lw = 2.0,
    xlims = xlims, 
    xticks = xticks, 
    fill = (zero_a, 0.0, a_color)
)

plot!(
    fig1_Ca[3], sol, vars = [:b], label = "",
    ylabel = "norm [TREK1]", xlabel = "",
    c = b_color, lw = 2.0, 
    xlims = xlims, 
    xticks = xticks, 
    legend = :topright,
    fill = (zero_a, 0.0, a_color)
)

plot!(
    fig1_Ca[4], sol, vars = [:v], label = "",
    ylabel = "$(vt) (mV)", xlabel = "time (ms)",
    c = v_color, lw = 2.0, 
    xlims = xlims, xticks = xticks
)

fig1_Cb = plot(xlims = (0.0, 1.25), ylims = (-1.2, 0.3))
hline!(fig1_Cb, [0.0], c = :red, linestyle = :dot, label = "")
hspan!(fig1_Cb, [0, 0.6], alpha = 0.4, c = :blue, label = "")
hspan!(fig1_Cb, [-20, 0.0], alpha = 0.4, c = :red, label = "")
plot!(fig1_Cb, 
    c_rng, a_rng, label = "",
    xlabel = "[Ca] (mM)", ylabel = "$(delta)[cAMP](norm/s)", 
    c = a_color, lw = 2.0, linestyle = :dash, 
    title = "[Ca] induced [cAMP] decay", titlefontsize = 10.0,
    fill = (0, 0.2, a_color)
)
plot!(fig1_Cb, [zero_c], [0.0], label = "[Ca] null = $(round(zero_c, digits = 2))mM",
    c = :purple, markersize = 4.0, seriestype = :scatter, )


fig1_Cc = plot(xlims = (0.0, 1.25), ylims = (-1.2, 0.3))
hline!(fig1_Cc, [0.0], c = :red, linestyle = :dot, label = "")
hspan!(fig1_Cc, [0, 0.6], alpha = 0.4, c = :blue, label = "")
hspan!(fig1_Cc, [-2.0, 0.0], alpha = 0.4, c = :red, label = "")
plot!( fig1_Cc,
    a_rng_2, b_rng, label = "", 
    xlabel = "norm [cAMP]", ylabel = "$(delta)[TREK1](norm/s)",
    title = "[cAMP] induced [TREK1] decay", titlefontsize = 10.0,
    c = b_color, lw = 2.0, linestyle = :dash, 
    fill = (0, 0.2, b_color)
)
plot!(fig1_Cc, [zero_a], [0.0], label = "[cAMP] null = $(round(zero_a, digits = 2))",
    c = :red, markersize = 4.0, seriestype = :scatter, )

#fig1_Cc = plot(c_full, b_full, xlabel = "[Ca]", ylabel = "[TREK1]")
fig1_C = plot(fig1_Ca, fig1_Cb, fig1_Cc, layout = grid(1,3, widths = [0.6, 0.2, 0.2]), grid = false)
title!(fig1_C[1], "C", titlepos = :left);

In [ ]:
fig1 = plot(fig1_A, fig1_B, fig1_C, layout = grid(3, 1, heights = [0.2, 0.3, 0.5]), size = (1000, 1000))

In [ ]:
savefig(fig1, "$(figure_path)/Figure1_ModelDynamics.png")

In [ ]:
import RetinalChaos.timescale_analysis

In [ ]:
#Conducting the analysis in a single function you can use the imported function timescale_analysis
vt_full = map(t -> sol(t)[1], sol.t)
#Mode 1 passes actual values for average and std
ts_analysis = timescale_analysis(vt_full; dt = dt, mode = 1, verbose = 1);
#Mode 2 passes back the lists
ts_lists = timescale_analysis(vt_full; dt = dt, mode = 2);

In [ ]:
println(ts_lists[3])